In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
import bempp.api.shapes.fractals

In [2]:
h = 0.05
dist = 3.0

sphere1 = bempp.api.shapes.sphere(h=h, origin=((dist/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(h=h, origin=(-((dist/2) + 1), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

for N_wn in [100]:

    for ubound_wn in [15, 20]:
        lbound_wn = 0
        quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
        results = []
        for index2, point2 in enumerate(quad_points):

            wavenumber = -1j * np.log(point2)

            slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
            slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
            slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
            slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

            mat11 = slp11.weak_form().A
            mat12 = slp12.weak_form().A
            mat21 = slp21.weak_form().A
            mat22 = slp22.weak_form().A
            mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
            mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

            mat = np.block([[mat11,mat12],[mat21,mat22]])  
            mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
            mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

            inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                                  [mat21_zero, mat22_inv]])  

            combined_inverse_free = inv_free_mat2.dot(mat)
            sign, logdet = slogdet(combined_inverse_free)

            results.append(logdet)
        print(N_wn, np.trapz(-np.array(results) / quad_points, quad_points) / (2 * np.pi))

100 0.0010190449953696582
100 0.0010199995157284523


In [ ]:
20 0.001029159937793544
20 0.0010291110604895611
20 0.0010340846191759226
20 0.0012684218805917372
30 0.0010231758573272338
30 0.0010231436216319465
30 0.0010264021587123842
30 0.0011799334678290168
40 0.0010211610825080011
40 0.0010211370350139523
40 0.0010235600492248147
40 0.0011377243560753306



In [ ]:
h = 0.1
dist = 1.5
N_wn = 20

sphere1 = bempp.api.shapes.sphere(h=h, origin=((dist/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(h=h, origin=(-((dist/2) + 1), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)


lbound_wn = 0
ubound_wn = 20
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
results = []
for index2, point2 in enumerate(quad_points):

    wavenumber = -1j * np.log(point2)

    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
    mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

    inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                          [mat21_zero, mat22_inv]])  

    combined_inverse_free = inv_free_mat2.dot(mat)
    sign, logdet = slogdet(combined_inverse_free)

    results.append(logdet)
print(np.trapz(-np.array(results) / quad_points, quad_points) / (2 * np.pi))

In [ ]:
%matplotlib inline
from matplotlib.pyplot import figure

fig = figure()
ax1 = fig.add_subplot(121)
ax1.set_xlabel('1j * k')
ax2 = fig.add_subplot(122)
ax2.set_xlabel('exp(-k)')
ax1.semilogy(-np.log(quad_points), -np.array(results))
ax2.plot(quad_points, -np.array(results))
fig.tight_layout()

In [ ]:
results

In [ ]:
-np.array(results)/quad_points

In [ ]:
quad_points

In [ ]:
-2.741140647799512e-13/1.38879439e-11

In [ ]:
3.0642155479654305e-13/2.06115362e-09

In [ ]:
0.04372859968911746

In [ ]:
CasE_05 = [0.044142920889365764, 0.04378745428754526, 0.043307825800813826, 0.0425274735585427] 
CasE_15 = [0.004684510596741076, 0.004668668075048978, 0.0046463126764854845, 0.004608327156406329] 
CasE_30 = [0.001029159937793544, 0.0010266636526601919, 0.0010231501929812861, 0.0010171903056843653]

In [ ]:
Rel_err_05 = [abs(CasE_05[0] - CasE_05[1])/CasE_05[0], abs(CasE_05[0] - CasE_05[2])/CasE_05[0], abs(CasE_05[0] - CasE_05[3])/CasE_05[0]]
Rel_err_15 = [abs(CasE_15[0] - CasE_15[1])/CasE_15[0], abs(CasE_15[0] - CasE_15[2])/CasE_15[0], abs(CasE_15[0] - CasE_15[3])/CasE_15[0]]
Rel_err_30 = [abs(CasE_30[0] - CasE_30[1])/CasE_30[0], abs(CasE_30[0] - CasE_30[2])/CasE_30[0], abs(CasE_30[0] - CasE_30[3])/CasE_30[0]]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))
ax1.plot([0.1, 0.15, 0.2], Rel_err_05,'*-')
ax1.set_xlabel('Grid size $h$', fontsize = 20)
ax1.set_ylabel('Relative error', fontsize = 15)
ax1.set_title('dist = 0.5', fontsize = 20)

ax2.plot([0.1, 0.15, 0.2], Rel_err_15,'*-')
ax2.set_xlabel('Grid size $h$', fontsize = 20)
ax2.set_ylabel('Relative error', fontsize = 15)
ax2.set_title('dist = 1.5', fontsize = 20)

ax3.plot([0.1, 0.15, 0.2], Rel_err_15,'*-')
ax3.set_xlabel('Grid size $h$', fontsize = 20)
ax3.set_ylabel('Relative error', fontsize = 15)
ax3.set_title('dist = 3.0', fontsize = 20)


fig.tight_layout()
fig.suptitle('Relative error in the scalar case', fontsize = 25)
fig.subplots_adjust(top=0.85)
fig.savefig('CasE_scalar.pdf')